In [1]:
import cv2
import numpy as np

In [2]:
import tensorflow as tf
gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
  try:
    # Currently, memory growth needs to be the same across GPUs
    for gpu in gpus:
      tf.config.experimental.set_memory_growth(gpu, True)
    logical_gpus = tf.config.experimental.list_logical_devices('GPU')
    print(len(gpus), "Physical GPUs,", len(logical_gpus), "Logical GPUs")
  except RuntimeError as e:
    # Memory growth must be set before GPUs have been initialized
    print(e)

1 Physical GPUs, 1 Logical GPUs


In [3]:
from tensorflow.keras.models import model_from_json

In [4]:
from tensorflow.keras.preprocessing import image

In [5]:
with open(r"..\Saved Model\fer_emopy.json", "r") as m:
    model = model_from_json(m.read())
    model.load_weights(r'..\Saved Model\fer_emopy.h5')

In [6]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 46, 46, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 46, 46, 64)        36928     
_________________________________________________________________
batch_normalization (BatchNo (None, 46, 46, 64)        256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 23, 23, 64)        0         
_________________________________________________________________
dropout (Dropout)            (None, 23, 23, 64)        0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 23, 23, 128)       73856     
_________________________________________________________________
batch_normalization_1 (Batch (None, 23, 23, 128)       5

In [7]:
from mtcnn import MTCNN

Using TensorFlow backend.


In [8]:
# from tensorflow.python.client import device_lib
# len(device_lib.list_local_devices())

In [31]:
capture  = cv2.VideoCapture(0)

In [14]:
ret, frame = capture.read()

In [15]:
test_img = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)

In [16]:
import matplotlib.pyplot as plt

In [1]:
plt.imshow(test_img)

In [18]:
detector = MTCNN()
faces = detector.detect_faces(test_img)
faces

[{'box': [271, 174, 140, 185],
  'confidence': 0.9999990463256836,
  'keypoints': {'left_eye': (313, 245),
   'right_eye': (378, 247),
   'nose': (343, 283),
   'mouth_left': (315, 316),
   'mouth_right': (373, 317)}},
 {'box': [598, 362, 42, 56],
  'confidence': 0.9987214207649231,
  'keypoints': {'left_eye': (609, 385),
   'right_eye': (630, 382),
   'nose': (620, 394),
   'mouth_left': (615, 407),
   'mouth_right': (630, 405)}}]

In [19]:
x,y,w,h = tuple(faces[0]['box'])

In [2]:
plt.imshow(test_img[y:y+h, x:x+w])

In [21]:
gr_img = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)

In [3]:
plt.imshow(gr_img, cmap=plt.cm.gray)

In [23]:
gray_face = gr_img[y:y+h, x:x+w]
gray_face = cv2.resize(gray_face, (48,48))
gray_img = image.img_to_array(gray_face)
gray_img = np.expand_dims(gray_img, axis = 0)
pred = model.predict(gray_img)

In [24]:
class_dict = {0: 'angry',
              1: 'disgust',
              2: 'fear',
              3: 'happy',
              4: 'neutral',
              5: 'sad',
              6: 'surprise'}

In [25]:
emotion = class_dict[np.argmax(pred[0])]
emotion

'happy'

In [6]:
cv2.putText(test_img, emotion, (int(x)+10, int(y)+10), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 3)
cv2.rectangle(test_img, tuple(faces[0]['box']), (0,255,0), thickness=2)
plt.imshow(test_img)

In [27]:
face_boxes = [tuple(face['box']) for face in faces]
for face_box in face_boxes:
    cv2.rectangle(test_img, face_box, (0,255,0), thickness=2)

In [7]:
plt.imshow(test_img);

In [32]:
while(capture.isOpened()):
    # Capture frame-by-frame
    ret, frame = capture.read()

    # Our operations on the frame come here
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    
    faces = detector.detect_faces(rgb)
    face_boxes = [tuple(face['box']) for face in faces]
    for face_box in face_boxes:
        cv2.rectangle(frame, face_box, (0,255,0), thickness=2)
        x, y, w, h = face_box
        gray_face = gray[y:y+h, x:x+w]
        gray_face = cv2.resize(gray_face, (48,48))
        gray_img = image.img_to_array(gray_face)
        gray_img = np.expand_dims(gray_img, axis = 0)
#         gray_img /= 255
        pred = model.predict(gray_img)
        emotion = class_dict[np.argmax(pred[0])]
        cv2.putText(frame, emotion, (int(x), int(y)), cv2.FONT_HERSHEY_SIMPLEX, 1, (0,0,255), 2)
    # Display the resulting frame
    cv2.imshow('frame', frame)
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break
# Run the cell below to stop capturing 

In [33]:
capture.release()
cv2.destroyAllWindows()